In [46]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *
from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging

import random
import time
import datetime

In [47]:
### THIS df GENERETE THE WHOLE DATA OF OUR CUSTOMERS TO GET SEGMENTED LATER
query = """
SELECT
    last_order_city AS city,
    source_id AS customer_id,
    recency_days AS recency_days,
    DATEDIFF(day, first_submit_date, GETDATE()) AS alive_days,
    CASE WHEN microsegment_name IS NULL THEN 'NO' ELSE microsegment_name END AS microsegment,
    CASE WHEN business_type IS NULL THEN 'NO' ELSE business_type END AS segment,
    CASE 
        WHEN is_kam_audited = 1 THEN 'KAM' 
        WHEN is_graduated = 1 THEN 'Grads'
        WHEN salesperson_mail <> 'No value' THEN 'Farming'
    ELSE 'Other' END AS canal--,
    --is_graduated AS grad,
    --is_kam_audited AS kam,
    --CASE WHEN salesperson_mail <> 'No value' THEN 1 ELSE 0 END AS is_farming
FROM dpr_customer.obt_customer

WHERE recency_days <= 180
"""
df = run_read_dwd_query(query)

In [48]:
card_ids = (328555,328623,328538,328532,328542,280014,279981,279987,280041,280016,238287,203020,203019,203017,-202917,-202908,-202950,-202941,-202919)

In [49]:
### THIS df GENERETE THE INFO OF THE CARD_IDS
query = """
SELECT 
    s.identifier_value AS city,
    dc.source_id AS customer_id,
    dp.card_id,
    SUM(CASE WHEN DATE(fs.order_close_date) >= DATE_TRUNC('day', date(CURRENT_DATE) - interval '7 day') THEN 1 ELSE 0 END) AS churn
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND dp.card_id IN {skus}
    AND DATE(fs.order_close_date) <= DATE_TRUNC('day', date(CURRENT_DATE) - interval '7 day')
    AND DATE(fs.order_close_date) >= DATE_TRUNC('day', date(CURRENT_DATE) - interval '60 day')
GROUP BY 1,2,3
HAVING churn > 0
""".format(skus=card_ids)
df2 = run_read_dwd_query(query)

In [50]:
### THIS df READS A SHEET TO BRING THE CURRENT SEGMENTS FOR THE CURRENT YEAR
df_segments = GoogleSheet("1xBW_HK4Gc1z13KbZ9xfC5WF6ozpSlI5dANnSLROTpw4")
df_segments = df_segments.get_as_dataframe('INFO')
df_segments['customer_segment_id'] = df_segments['customer_segment_id'].astype(int)

In [51]:
def cayena():
    df_cayena=GoogleSheet("1tQZXQaLQqUF-Y5Ue8yQttcy-i5uiqPSSJpzpm1mS5gY")
    df_cayena = df_cayena.get_as_dataframe('Dados Pedidos')

    customers_cayena = df_cayena.loc[df_cayena['Cadastro Frubana?'] == 'Sim','ID'].unique().tolist()
    return customers_cayena

In [52]:
def rest_kams_extra_inv(city,card_id):
    customers =  df.loc[
                (df.city == city) &
                (df.segment.isin(['Restaurante'])) &
                (~df.microsegment.isin(['ALL'])) &
                (~df.canal.isin(['ALL'])) &
                (df.alive_days <= 10000) &
                (df.recency_days <= 60) &
                (~df.customer_id.isin(df2.loc[df2.card_id == card_id].customer_id.unique())) 
            ].customer_id.unique().tolist()
    customers_kam =  df.loc[
                (df.city == city) &
                (~df.segment.isin(['ALL'])) &
                (~df.microsegment.isin(['ALL'])) &
                (df.canal.isin(['KAM'])) &
                (df.alive_days <= 10000) &
                (df.recency_days <= 60) &
                (~df.customer_id.isin(df2.loc[df2.card_id == card_id].customer_id.unique())) 
            ].customer_id.unique().tolist()
    return list(set(customers + customers_kam))

In [53]:
def update_segments(city,customers,customerSegmentId):
    url = f"https://{city}.frubana.com/api/v1/segments/massive/update?api-key=00_growth-team-zzz-qqq_001"

    payload = json.dumps({
    "customerSegmentId": customerSegmentId,
    "customersIds": customers
    })
    headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.text

In [55]:
for city in df.city.unique():
    for card_id in df2.loc[df2.city == city].card_id.unique():
        if city == 'SPO':
            customerSegmentId = df_segments.loc[(df_segments.city == city) & (df_segments.description == f'{city}_{card_id}'),'customer_segment_id'].values[0]
            customers = list(set(rest_kams_extra_inv(city,card_id)) - set(cayena()))
        else:
            customerSegmentId = df_segments.loc[(df_segments.city == city) & (df_segments.description == f'{city}_{card_id}'),'customer_segment_id'].values[0]
            customers = rest_kams_extra_inv(city,card_id)
            
        try:
            response = update_segments(city,customers,int(customerSegmentId))
            message = f'IT RUNS OK FOR {city} FOR {city}_{card_id} -> ' + response + ' ' + str(customerSegmentId) + '|| TOTAL CUSTOMERS:' + str(len(customers))
            print(message)
        except:
            message = f"IT WAS AN ERROR IN {city} FOR {city}_{card_id}, ID: {customerSegmentId} || ERROR: {str(e)}"
            print(message)

IT RUNS OK FOR SPO FOR SPO_-202908 -> The massive update process of Customer Segment was created 643051|| TOTAL CUSTOMERS:19931
IT RUNS OK FOR SPO FOR SPO_-202919 -> The massive update process of Customer Segment was created 642901|| TOTAL CUSTOMERS:19853
IT RUNS OK FOR SPO FOR SPO_-202941 -> The massive update process of Customer Segment was created 642851|| TOTAL CUSTOMERS:19832
IT RUNS OK FOR SPO FOR SPO_-202917 -> The massive update process of Customer Segment was created 642951|| TOTAL CUSTOMERS:20036
IT RUNS OK FOR SPO FOR SPO_-202950 -> The massive update process of Customer Segment was created 643001|| TOTAL CUSTOMERS:19689
IT RUNS OK FOR BHZ FOR BHZ_203017 -> The massive update process of Customer Segment was created 642902|| TOTAL CUSTOMERS:5485
IT RUNS OK FOR BHZ FOR BHZ_203019 -> The massive update process of Customer Segment was created 643251|| TOTAL CUSTOMERS:5471
IT RUNS OK FOR BHZ FOR BHZ_203020 -> The massive update process of Customer Segment was created 643102|| TOT